In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
def load_trained_model(filename='enhanced_subway_model_fast.pkl'):
    """훈련된 모델 로드"""
    try:
        model_data = joblib.load(filename)
        print(f"✅ 모델 로드 완료: {filename}")
        return model_data
    except Exception as e:
        print(f"❌ 모델 로드 실패: {str(e)}")
        try:
            model_data = joblib.load('../result/enhanced_subway_model_fast.pkl')
            print(f"✅ 모델 로드 완료: ../result/enhanced_subway_model_fast.pkl")
            return model_data
        except:
            print("❌ 대체 경로에서도 모델 로드 실패")
            return None

In [ ]:
class ModelInsights:
    def __init__(self, model_data):
        self.best_model = model_data['best_model']
        self.feature_engineer = model_data['feature_engineer']
        self.selected_features = model_data['selected_features']
        self.results = model_data['results']
        self.feature_importance = model_data['feature_importance']
        # 시각화용 데이터
        self.X_test = model_data['test_data']['X_test']
        self.y_test = model_data['test_data']['y_test']
        self.test_data = model_data['test_data']['test_data_full']
    
    def plot_results(self):
        """결과 시각화 (원본과 동일)"""
        print("\n📊 결과 시각화 생성 중...")
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle('고도화된 지하철 혼잡도 예측 모델 결과', fontsize=16, fontweight='bold')
        
        # 1) 모델 성능 비교
        model_names = list(self.results.keys())
        mae_scores = [self.results[name]['mae'] for name in model_names]
        r2_scores = [self.results[name]['r2'] for name in model_names]
        
        axes[0, 0].bar(model_names, mae_scores, color='skyblue', alpha=0.7)
        axes[0, 0].set_title('모델별 MAE 비교')
        axes[0, 0].set_ylabel('Mean Absolute Error')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        axes[0, 1].bar(model_names, r2_scores, color='lightgreen', alpha=0.7)
        axes[0, 1].set_title('모델별 R² 비교')
        axes[0, 1].set_ylabel('R² Score')
        axes[0, 1].tick_params(axis='x', rotation=45)
        
        # 2) 특성 중요도 (상위 15개)
        if self.feature_importance is not None:
            top_features = self.feature_importance.head(15)
            axes[0, 2].barh(top_features['feature'][::-1], top_features['importance'][::-1])
            axes[0, 2].set_title('특성 중요도 (상위 15개)')
            axes[0, 2].set_xlabel('중요도')
        
        # 3) 예측 vs 실제 (최고 모델)
        if self.best_model is not None:
            y_pred = self.best_model.predict(self.X_test)
            
            # 샘플링 (시각화 최적화)
            if len(self.y_test) > 10000:
                sample_idx = np.random.choice(len(self.y_test), 10000, replace=False)
                y_test_sample = self.y_test.iloc[sample_idx]
                y_pred_sample = y_pred[sample_idx]
            else:
                y_test_sample = self.y_test
                y_pred_sample = y_pred
            
            axes[1, 0].scatter(y_test_sample, y_pred_sample, alpha=0.5, s=1)
            axes[1, 0].plot([y_test_sample.min(), y_test_sample.max()], 
                           [y_test_sample.min(), y_test_sample.max()], 'r--', lw=2)
            axes[1, 0].set_xlabel('실제 혼잡도')
            axes[1, 0].set_ylabel('예측 혼잡도')
            axes[1, 0].set_title('예측 vs 실제 (최고 모델)')
            
            # 4) 잔차 분포
            residuals = y_test_sample - y_pred_sample
            axes[1, 1].hist(residuals, bins=50, alpha=0.7, color='orange')
            axes[1, 1].set_xlabel('잔차 (실제 - 예측)')
            axes[1, 1].set_ylabel('빈도')
            axes[1, 1].set_title('잔차 분포')
            axes[1, 1].axvline(0, color='red', linestyle='--')
            
            # 5) 시간대별 성능
            test_data_with_pred = self.test_data.copy()
            test_data_with_pred['predictions'] = self.best_model.predict(self.X_test)
            test_data_with_pred['residuals'] = abs(test_data_with_pred['congestion'] - test_data_with_pred['predictions'])
            
            hourly_mae = test_data_with_pred.groupby('hour')['residuals'].mean()
            axes[1, 2].plot(hourly_mae.index, hourly_mae.values, marker='o', linewidth=2)
            axes[1, 2].set_title('시간대별 예측 오차 (MAE)')
            axes[1, 2].set_xlabel('시간')
            axes[1, 2].set_ylabel('평균 절대 오차')
            axes[1, 2].grid(True, alpha=0.3)
            axes[1, 2].set_xticks(range(0, 24, 2))
        
        plt.tight_layout()
        try:
            plt.savefig('../result/enhanced_model_results.png', dpi=300, bbox_inches='tight')
        except:
            plt.savefig('enhanced_model_results.png', dpi=300, bbox_inches='tight')
        plt.show()
    
    def generate_insights(self):
        """개선된 모델 인사이트 (원본과 동일)"""
        print("\n" + "=" * 60)
        print("🚀 고도화된 모델 인사이트")
        print("=" * 60)
        
        best_model_name = min(self.results.keys(), key=lambda x: self.results[x]['mae'])
        best_result = self.results[best_model_name]
        
        print(f"\n🏆 최고 성능 모델: {best_model_name}")
        print(f"  📊 MAE: {best_result['mae']:.3f}")
        print(f"  📊 RMSE: {best_result['rmse']:.3f}")
        print(f"  📊 R²: {best_result['r2']:.3f}")
        
        # 성능 개선 분석
        print(f"\n📈 모델 개선 효과:")
        if best_result['r2'] > 0.8:
            print("  ✅ 우수한 예측 성능 달성")
        elif best_result['r2'] > 0.6:
            print("  ⭐ 양호한 예측 성능")
        else:
            print("  🔧 추가 개선 필요")
        
        # 특성 중요도 인사이트
        if self.feature_importance is not None:
            print(f"\n🎯 핵심 예측 요인 (상위 5개):")
            for i, (_, row) in enumerate(self.feature_importance.head(5).iterrows()):
                print(f"  {i+1}. {row['feature']}: {row['importance']:.3f}")
            
            # 특성 유형별 분석
            importance_by_type = {}
            for _, row in self.feature_importance.iterrows():
                feature = row['feature']
                if any(x in feature for x in ['extreme_', 'heavy_', 'strong_']):
                    importance_by_type['극한기상'] = importance_by_type.get('극한기상', 0) + row['importance']
                elif any(x in feature for x in ['interaction', '_x_', 'temp_humidity']):
                    importance_by_type['상호작용'] = importance_by_type.get('상호작용', 0) + row['importance']
                elif any(x in feature for x in ['lag_', '_ma_', 'shift']):
                    importance_by_type['시차특성'] = importance_by_type.get('시차특성', 0) + row['importance']
                elif any(x in feature for x in ['hour', 'day', 'time_period', 'season']):
                    importance_by_type['시간특성'] = importance_by_type.get('시간특성', 0) + row['importance']
                else:
                    importance_by_type['기본특성'] = importance_by_type.get('기본특성', 0) + row['importance']
            
            print(f"\n📊 특성 유형별 중요도:")
            for feature_type, importance in sorted(importance_by_type.items(), key=lambda x: x[1], reverse=True):
                print(f"  {feature_type}: {importance:.3f}")
        
        print(f"\n💡 실무 활용 제안:")
        print("  🎯 극한 기상 조건에서의 혼잡도 예측 정확도 향상")
        print("  🎯 세분화된 시간대별 맞춤형 운영 전략 수립")
        print("  🎯 기상-시간 상호작용을 고려한 동적 배차 계획")
        print("  🎯 특성 중요도 기반 핵심 요인 모니터링")

In [ ]:
# 모델 로드
model_data = load_trained_model()

if model_data is not None:
    # 인사이트 분석 객체 생성
    insights = ModelInsights(model_data)
    print("✅ 인사이트 분석 준비 완료")
else:
    print("❌ 모델을 로드할 수 없어 인사이트 분석을 진행할 수 없습니다.")

In [ ]:
if model_data is not None:
    try:
        insights.plot_results()
    except Exception as e:
        print(f"⚠️ 시각화 오류: {str(e)}")

In [ ]:
insights.generate_insights() if insights else print("❌ 모델 로드 실패")